## Service Pading and Indexed


In [1]:
# import plaidml.keras
# plaidml.keras.install_backend()
import nltk
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import csv
import h5py
from pandas import HDFStore, read_hdf
import pandas as pd

#from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence

np.random.seed(0)
%matplotlib inline

Using TensorFlow backend.


## 1. Read from H5

In [2]:
AllData = read_hdf('../Data/RandomSplittedByCatagories9.h5', key='AllData')
TrainServices = read_hdf('../Data/RandomSplittedByCatagories9.h5', key='Train')
TestServices = read_hdf('../Data/RandomSplittedByCatagories9.h5', key='Test')
print(TrainServices.shape)
print(TestServices.shape)

(8123, 2)
(2061, 2)


In [3]:
X = AllData['Service Desciption']
Y = AllData['Service Classification']

Train_X = list(TrainServices['Service Desciption'])
Train_Y = list(TrainServices['Service Classification'])

Test_X = list(TestServices['Service Desciption'])
Test_Y = list(TestServices['Service Classification'])

In [4]:
Train_X[0]

'The Federal Reserve Bank of St. Louis is one of 12 regional Reserve banks in the Fed System. The Research Division of the bank looks to promote quality economic research and contribute to economic policy discussions. In support of this goal, it has created APIs that let developers access the data stored on its web site. The FRED API lets users query the Federal Reserve Economic Data (FRED) and Archival Federal Reserve Economic Data (ALFRED) databases to retrieve specific data. The requested data can be customized according to data source, release, category, series, and other preferences. The API uses RESTful calls and responses are formatted in XML.'

## 2. Text to index by Glove 

In [5]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf-8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

### Get word_to_index, index_to_word from Glove

In [6]:
t = Tokenizer()
t.fit_on_texts(X)
vocab_size = len(t.word_index) + 1
print("vocab size in corpus: " + str(vocab_size))

word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('../Data/glove.6B.200d.txt')

print("vocab size in Glove:", len(word_to_index))

vocab size in corpus: 23560
vocab size in Glove: 400000


In [7]:
WordsNotInGlove = []

for k, v in t.word_index.items():
    if k not in word_to_index:
        WordsNotInGlove.append(k)
        
print("length:", len(WordsNotInGlove)) 

length: 8619


In [8]:
len(WordsNotInGlove)

8619

In [9]:
t.word_index["nasa's"]

3339

### use Glove5b to Index dataset

In [10]:
def glove_word_to_index(X):

    allservices = []

    for s in X:

        words = text_to_word_sequence(s)

        service = []

        for w in words:

            if w in word_to_index:
                index = word_to_index[w]
                service.append(index)

        allservices.append(service)

    return allservices

In [11]:
indexed_Train_X = glove_word_to_index(Train_X)
indexed_Test_X = glove_word_to_index(Test_X)

In [12]:
print(Train_X[4])
print(indexed_Train_X[4])

The 3PlayMedia Advanced Timeshift API allows developers to caption timing changes in edited video sections. It supports advanced parameters for extracting in/out points from caption files. The API is applicable to any completed file in the 3PlayMedia system. 3play Media is a provider of captioning and transcription solutions for a variety of media platforms including online video, media & entertainment, education, e-learning, and market research.
[357266, 46991, 359990, 57459, 52074, 122453, 360915, 91574, 360028, 96737, 188481, 133578, 378547, 323172, 193716, 348224, 46991, 277191, 151349, 143138, 188481, 272930, 287518, 154323, 91574, 148107, 357266, 57459, 192973, 57833, 360915, 57170, 106328, 148092, 188481, 357266, 350362, 239464, 192973, 43010, 294047, 268046, 91576, 54718, 363856, 337177, 151349, 43010, 376136, 268046, 239464, 286285, 188839, 270193, 378547, 239464, 137889, 133717, 132033, 219115, 54718, 234840, 306262]


In [13]:
print(Test_X[4])
print(indexed_Test_X[4])

The Intrinio Algiers Stock Exchange Prices API offers end of day (EOD) prices from the Algiers Stock Exchange in Algeria. The API returns the most recent data point for a selected identifier such as stock market index symbol, and CIK ID. Additionally, the API offers professional-grade historical data, and lists of securities. Intrinio offers payment plans for individuals, startups, developers, and enterprises. Intrinio is a provider of financial services that delivers data collection, data entry, and data analysis tools.
[357266, 51254, 343465, 142368, 292136, 57459, 268194, 136979, 268046, 117874, 138180, 292136, 154323, 357266, 51254, 343465, 142368, 188481, 51245, 357266, 57459, 307037, 357266, 250392, 302801, 117493, 287479, 151349, 43010, 323929, 186400, 346373, 60665, 343465, 234840, 189109, 349979, 54718, 101135, 186309, 46226, 357266, 57459, 268194, 293041, 165534, 179550, 117493, 54718, 223663, 268046, 323216, 268194, 279342, 286125, 151349, 189275, 341905, 122453, 54718, 1378

## 3. Padding to same length

In [14]:
maxLen = 110

In [15]:
indexed_padded_Train_X = pad_sequences(indexed_Train_X, maxlen=maxLen, padding='post')
indexed_padded_Test_X = pad_sequences(indexed_Test_X, maxlen=maxLen, padding='post')

## 4. Category to one-hot

In [16]:
categories = list(set(Y))
categories_num = len(categories)
index = list(range(len(categories)))
index_to_category = dict(zip(index, categories)) 
category_to_index = dict(zip(categories, index))

In [17]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [18]:
Y_Index = np.asarray([category_to_index[k] for k in Y])
Y_one_hot = convert_to_one_hot(Y_Index, 50)
Y_one_hot.shape

(10184, 50)

In [25]:
Y_Index[1]

1

In [26]:
Y_one_hot[1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
Train_Y_Index = np.asarray([category_to_index[k] for k in Train_Y])
Train_Y_one_hot = convert_to_one_hot(Train_Y_Index, 50)
print(Train_Y_one_hot.shape)

Test_Y_Index = np.asarray([category_to_index[k] for k in Test_Y])
Test_Y_one_hot = convert_to_one_hot(Test_Y_Index, 50)
print(Test_Y_one_hot.shape)

(8123, 50)
(2061, 50)


## 5. Save category indexes

In [21]:
import json
with open('../Data/category_to_index.json', 'w') as fp:
    json.dump(category_to_index, fp)
    
with open('../Data/index_to_category.json', 'w') as fp:
    json.dump(index_to_category, fp)

## 6. Save Datasets (np to h5)

In [22]:
h5f = h5py.File('../Data/SplittedPaddedIndexedServiceDataset.h5', 'w')
h5f.create_dataset('indexed_padded_Train_X', data=indexed_padded_Train_X)
h5f.create_dataset('Train_Y_one_hot', data=Train_Y_one_hot)   
h5f.create_dataset('indexed_padded_Test_X', data=indexed_padded_Test_X)
h5f.create_dataset('Test_Y_one_hot', data=Test_Y_one_hot)
h5f.close()

In [21]:
h5f = h5py.File('../Data/SplittedPaddedIndexedServiceDataset.h5','r')
indexed_padded_Train_X = h5f['indexed_padded_Train_X'][:]
Train_Y_one_hot = h5f['Train_Y_one_hot'][:]
indexed_padded_Test_X = h5f['indexed_padded_Test_X'][:]
Test_Y_one_hot = h5f['Test_Y_one_hot'][:]
print(indexed_padded_Train_X.shape, Train_Y_one_hot.shape)
print(indexed_padded_Test_X.shape, Test_Y_one_hot.shape)
h5f.close()

(8123, 110) (8123, 50)
(2061, 110) (2061, 50)
